In [4]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import pylab
import glob
import time
%matplotlib inline

/Users/Kim/Software/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## HDI Mosaic and Bias-Correction Procedure

### The following *incomplete* code is written as a starting point to demonstrate two goals: 

1) convert a multi-extension (4 amplifier) HDI image into a regular, single extension FITS image,  
and  
2) correct for the bias drift and subtract the bias signal from HDI images.

**Your tasks tonight:**  
(1) Comment the code below so it makes sense to you  
(2) Finish the code so it runs successfully on a test image (re-create what we saw in class!)  
(3) Go to the benchmarking notebook and match your pseudocode to the functions within (and test your computer/SD card!)  

**Your tasks this week:**  
(1) Create your own new bias-correction notebook, using the functions from the benchmarking notebook and writing   
(2) Apply your bias correction to Night 4 of data (both calibration frames and science frames) for your project  
(3) Save the newly-made mosaiced, bias-corrected images  
(4) Describe the how, what, and why of these reduction steps in your weekly research blog (due **5 pm Weds. 2/7**).

**What you will need for this notebook:**
* A single science image to test the mosaic part of the code (read_raw_fits) and the bias-correction (overscan_only_bias_correction)

**What you will need for the benchmarking notebook:**
* A stack of files (e.g., biases) to median combine.



In [1]:
# How big should our mosaic image (all four quadrants in one image) be? This will *include* the overscan regions.

# Initialize parameters for the final image size, below:

final_imx = 2056*2
final_imy = 2156*2

In [2]:
def read_raw_fits(im, newimx, newimy):
    """
    [add your own docstring and add comments to describe the purpose of this function, filling out templates below]

    Purpose: 
    
    
    
    Input:
    
    
    
    Returns:
    
    
    
    Example Usage:
    
    
    
    """
    
    # What is taking place in the following steps?
    # Answer:
    
    im_padded = np.zeros([newimx, newimy])
    im_all = fits.open(im)
    im_header = im_all[0].header
    
    
    
    # Some useful info:
    
    # im_padded[newimx/2:newimx, 0:newimy/2] places data in the upper left quadrant
    # im_all[4].data corresponds to the northeast quadrant of the image; needs to be inverted in Y
    
    # [(newimx/2):(newimx), newimy/2:(newimy)] places data in the upper right quadrant
    # im_all[3].data corresponds to the northwest quadrant of the image; needs to be inverted in X and Y
    
    # [0:newimx/2, 0:newimy/2] places data in lower left quadrant
    # im_all[2].data corresponds to the southeast quadrant of the image; needs no inversion
    
    # [0:newimx/2, newimy/2:newimy] places data in the lower right quadrant
    # im_all[1].data corresponds to the southwest quadrant of the image; needs to be inverted in X
    
    
    
    
    # What is taking place in the following four commands?
    # Answer:

    # northeast
    im_padded[newimx/2:newimx, 0:newimy/2] = np.flipud(im_all[4].data) # can also write im_all[4].data[::-1]
    
    # northwest
    im_padded[(newimx/2):(newimx), newimy/2:(newimy)] = np.fliplr(np.flipud(im_all[3].data)) # can also write im_all[3].data[::-1,::-1]
    
    # southeast
    im_padded[0:newimx/2, 0:newimy/2] = im_all[2].data
    
    # southwest
    im_padded[0:newimx/2, newimy/2:newimy] = np.fliplr(im_all[1].data)

    return im_padded, im_header
        

### In the cell below, try out the read_raw_fits function on a single test science image.

In [5]:
# complete this cell:

test_mosaic_im, test_mosaic_header = read_raw_fits(

### Now save the file you've just read as a new FITS file. Call it *test_mosaic.fits*:

In [6]:
# complete this: write your own code to save a new FITS file here



### Test the before/after FITS files by opening them in ds9. How did it work?

(Open the old one with File -> Open As -> Mosaic IRAF; the new one should open like a regular FITS file.)

### Answer:

## The bias-correction function

In [1]:
def overscan_only_bias_correct(fits_im, newimx, newimy):
    """
    Purpose: 
    For a given FITS image (fits_im), read in imframe and measure the mean overscan values for each of its quadrants.
    Then, apply the offsets to each quadrant: subtract the appropriate bias level from each quadrant in the FITS 
    image of interest.
        
    Input: 
    fits_im - raw science (or flat) frame to be corrected
    newimx - dimensions in x-axis 
    newimy - dimensions in y-axis 
    
    Returns: 
    corr_image - a bias-corrected version of the input science (or flat) image
    im_hdr - header of the input science (or flat) image
                
    Example Usage: 
    new_corrected_im, corrected_im_header = overscan_only_bias_correct('image.fits', x_image_size, y_image_size)
    """
    
    # First, make your input science (or flat) image into a four-quadrant mosaic. 
    # Finish the following line of code:
    imframe, imframe_hdr = read_raw_fits(

        
        
    
    # Measure the overscan levels for each quadrant: 
    
    # Reminder: [newimx/2:newimx, 0:newimy/2] => upper left quadrant
    overscan_ul = np.median(imframe[newimx/2:newimx, (newimy/2 - 108):newimy/2]) 
    
    # Reminder: [(newimx/2):(newimx), newimy/2:(newimy)] => upper right quadrant
    overscan_ur = np.median(imframe[(newimx/2):(newimx), newimy/2:(newimy/2 + 108)])
    
    # Reminder: [0:newimx/2, 0:newimy/2] => lower left quadrant
    overscan_ll = np.median(imframe[0:newimx/2, (newimy/2-108):newimy/2])

    # Reminder: [0:newimx/2, newimy/2:newimy] => lower right quadrant
    overscan_lr = np.median(imframe[0:newimx/2, (newimy/2):(newimy/2)+108])


        

    # Now make a new empty image to place each subtracted quadrant
    corr_im = np.zeros([newimx, newimy])

    # ul
    corr_im[newimx/2:newimx, 0:newimy/2] = imframe[newimx/2:newimx, 0:newimy/2]- overscan_ul

    # ur
    corr_im[(newimx/2):(newimx), newimy/2:(newimy)] = imframe[(newimx/2):(newimx), newimy/2:(newimy)] - overscan_ur

    # ll
    corr_im[0:newimx/2, 0:newimy/2] = imframe[0:newimx/2, 0:newimy/2] - overscan_ll

    # lr
    corr_im[0:newimx/2, newimy/2:newimy] = imframe[0:newimx/2, newimy/2:newimy] - overscan_lr
    
    return corr_im, imframe_hdr


## Run the bias-correction function on your test science image:

In [18]:
# Complete the following cell:

bias_corr_image_overscan, header = overscan_only_bias_correct(

In [19]:
# Write the bias-corrected file:

fits.writeto(

### Finally, open the original file (ds9 -> open as -> mosaic IRAF) and the new one. How do they compare? Was the bias level removed?

**Answer:**